In [1093]:
import pandas as pd
import re
test_df = pd.read_csv("testData.csv")

In [1094]:
# Remove English alphanumeric values
alpha_pat = "[a-zA-z0-9]"

test_df["text"] = test_df["text"].str.replace(alpha_pat, "", regex=True)

In [1095]:
# Handling দাঁড়ি problem
d = '।'
test_df['text'] = test_df["text"].str.replace(d, " ।", regex=True)

In [1096]:
bangla_numerals =     ["শুন্য", "এক", "দুই", "তিন", "চার", "পাঁচ", "ছয়", "সাত", "আট", "নয়", "দশ",
                      "এগারো", "বার", "তেড়", "চৌদ্দ", "পনের", "ষোল", "সতের", "আঠার", "উনিশ", "বিশ",
                      "একুশ", "বাইশ", "তেইশ", "চব্বিশ", "পঁচিশ", "ছাব্বিশ", "সাতাশ", "আঠাশ", "ঊনত্রিশ", "ত্রিশ",
                      "একত্রিশ", "বত্রিশ", "তেত্রিশ", "চৌত্রিশ", "পঁয়ত্রিশ", "ছত্রিশ", "সাঁইত্রিশ" , "আটত্রিশ", "ঊনচল্লিশ", "চল্লিশ",
                      "একচল্লিশ", "বিয়াল্লিশ", "তেতাল্লিশ", "চুয়াল্লিশ", "পঁয়তাল্লিশ", "ছেচল্লিশ", "সাতচল্লিশ", "আটচল্লিশ", "ঊনপঞ্চাশ", "পঞ্চাশ",
                      "একান্ন", "বায়ান্ন", "তিপ্পান্ন", "চুয়ান্ন", "পঞ্চান্ন", "ছাপ্পান্ন", "সাতান্ন", "আটান্ন", "ঊনষাট", "ষাট",
                      "একষট্টি", "বাষট্টি", "তেষট্টি", "চৌষট্টি", "পঁয়ষট্টি", "ছেষট্টি", "সাতষট্টি", "আটষট্টি", "ঊনসত্তর", "সত্তর",
                      "একাত্তর", "বাহাত্তর", "তিয়াত্তর", "চুয়াত্তর", "পঁচাত্তর", "ছিয়াত্তর", "সাতাত্তর", "আটাত্তর", "ঊনআশি", "আশি",
                      "একাশি", "বিরাশি", "তিরাশি", "চুরাশি", "পঁচাশি", "ছিয়াশি", "সাতাশি", "আটাশি", "ঊননব্বই", "নব্বই",
                      "একানব্বই", "বিরানব্বই", "তিরানব্বই", "চুরানব্বই", "পঁচানব্বই", "ছিয়ানব্বই", "সাতানব্বই", "আটানব্বই", "নিরানব্বই",
                       "একশো"]

In [1097]:
def num2bangla(input_number, context=False):
    if not str(input_number).isdigit():
        return "NotValid"
    #Context True Means if it is digit by digit
    if context:
        return ' '.join(bangla_numerals[int(digit)] for digit in str(input_number))

    input_number = int(input_number)

    if not 0 <= input_number <= 999999999:
        return "NotInRange"


    
    crore, input_number = divmod(input_number, 10000000)
    lakh, input_number = divmod(input_number, 100000)
    thousand, input_number = divmod(input_number, 1000)
    hundred, input_number = divmod(input_number, 100)
    tens, ones = divmod(input_number, 10)

    result = ""

    if crore:
        result += num2bangla(crore) + " কোটি "
    if lakh:
        result += num2bangla(lakh) + " লাখ"
    if thousand:
        result += " " + num2bangla(thousand) + " হাজার"
    if hundred:
        result += " " + num2bangla(hundred) + "শো"

    if (tens or ones) and input_number>=0 and input_number<=100:
        if result:
            result += " " 
        result += bangla_numerals[int(input_number)]

    return result.lstrip() or "শুন্য"

In [1098]:
def replace_numerals_with_words(text, context=False):
    words = text.split()
    modified_words = []

    for word in words:
        #handled 911
        #if word == '৯১১':
            #modified_words.append(num2bangla(word, True))
            #continue
            
        #handled the plus sign
        if word == '+':
            modified_words.append('যোগ')
            continue
        if '+' in word and len(str(word))>1:
            chars_to_ignore = '[()]'
            word = re.sub(chars_to_ignore, '', word)
            parts = word.split('+')
            modified_words.append(num2bangla(parts[0], context))
            modified_words.append("যোগ")
            if str(parts[1]).isdigit():
                modified_words.append(num2bangla(parts[1], context))
            continue
            
        #handles the equal sign
        if word == '=':
            modified_words.append('সমান')
            continue
        
        #handled date
        date = word.split('/')
        if len(date) == 3 and len(str(date[1])) == 2:
            modified_words.append(num2bangla(date[0][-2:], False))
            if date[1] == '০১':
                modified_words.append('জানুয়ারি')
            elif date[1] == '০২':
                modified_words.append('ফেব্রুয়ারি')
            elif date[1] == '০৩':
                modified_words.append('মার্চ')
            elif date[1] == '০৪':
                modified_words.append('এপ্রিল')
            elif date[1] == '০৫':
                modified_words.append('মে')
            elif date[1] == '০৬':
                modified_words.append('জুন')
            elif date[1] == '০৭':
                modified_words.append('জুলাই')
            elif date[1] == '০৮':
                modified_words.append('আগস্ট')
            elif date[1] == '০৯':
                modified_words.append('সেপ্টেম্বর')
            elif date[1] == '১০':
                modified_words.append('অক্টোবর')
            elif date[1] == '১১':
                modified_words.append('নভেম্বর')
            elif date[1] == '১২':
                modified_words.append('ডিসেম্বর')
            modified_words.append(num2bangla(date[2][:4], False))
            continue
        
        #handled double dots
        decimal = re.sub(re.escape('.'), '', word)
        if len(word.split('.')) == 3 and decimal.isnumeric():
            parts = word.split('.')
            modified_words.append(num2bangla(parts[0], False))
            modified_words.append("দশমিক")
            modified_words.append(num2bangla(parts[1], False))
            modified_words.append("দশমিক")
            modified_words.append(num2bangla(parts[2], False))
            continue
        
        #handled the "."
        decimal = re.sub(re.escape('.'), '', word)
        if '.' in word and decimal.isnumeric():
            parts = word.split('.')
            modified_words.append(num2bangla(parts[0], False))
            if str(parts[1]).isdigit():
                modified_words.append("দশমিক")
                modified_words.append(num2bangla(parts[1], True))
            continue
        
        #handle phone number
        if word.startswith('০'):
            z = 0
            if word[-1] == ',':
                z = 1
            chars_to_ignore = '[-,\.:;\'"!\?।()]'
            word = re.sub(chars_to_ignore, '', word)
            modified_words.append((num2bangla(word, context=True)+',' if z==1 else num2bangla(word, context=True)))
            continue

        #handled the ","
        comma = re.sub(re.escape(','), '', word)
        if( comma.isnumeric() and int(comma)>0):
            z = 0
            if word[-1] == ',':
                z = 1
            word = re.sub(re.escape(','), '', word);
            modified_words.append((num2bangla(word, context)+',' if z==1 else num2bangla(word, context)))
            continue
            
        #handled ১ম থেকে ১০ম
        z = ['১ম', '২য়', '৩য়', '৪র্থ', '৫ম', '৬ষ্ঠ', '৭ম', '৮ম', '৯ম', '১০ম']
        if word in z:
            if word == '১ম':
                modified_words.append('প্রথম')
                continue
            elif word == '২য়':
                modified_words.append('দ্বিতীয়')
                continue
            elif word == '৩য়':
                modified_words.append('তৃতীয়')
                continue
            elif word == '৪র্থ':
                modified_words.append('চতুর্থ')
                continue
            elif word == '৫ম':
                modified_words.append('পঞ্চম')
                continue
            elif word == '৬ষ্ঠ':
                modified_words.append('ষষ্ঠ')
                continue
            elif word == '৭ম':
                modified_words.append('সপ্তম')
                continue
            elif word == '৮ম':
                modified_words.append('অষ্টম')
                continue
            elif word == '৯ম':
                modified_words.append('নবম')
                continue
            elif word == '১০ম':
                modified_words.append('দশম')
                continue
        
        #handled ১লা থেকে ৪ঠা
        date = ['১লা', '২রা', '৩রা', '৪ঠা']
        if word in date:
            if word == '১লা':
                modified_words.append('পহেলা')
                continue
            elif word == '২রা':
                modified_words.append('দোসরা')
                continue
            elif word == '৩রা':
                modified_words.append('তেসরা')
                continue
            elif word == '৪ঠা':
                modified_words.append('চৌঠা')
                continue
        
        #handled if it is attached with a string
        else:
            temp_modified = []
            parts = re.split(r'(\d+)', word)
            for part in parts:
                if part.isnumeric():
                    part = num2bangla(part, context)
                elif part =='%':
                    part = 'শতাংশ' 
                temp_modified.append(part)
            modified_words.append(''.join(temp_modified))

    updated_text = ' '.join(modified_words)
    return updated_text

In [1099]:
#test_df = pd.read_csv("testData.csv", index_col="row_id_column_name")
print("Total number of test samples:", len(test_df))

print("Number of test samples containing Bangla numerals:", len(test_df[lambda x: x["text"].str.contains("[০১২৩৪৫৬৭৮৯]")]))
test_dff = test_df.copy(deep = True)
# Apply the 'replace_numerals_with_words' function to transform numerals in the filtered text
test_dff['text'] = test_dff['text'].apply(replace_numerals_with_words)

Total number of test samples: 27228
Number of test samples containing Bangla numerals: 9602


In [1100]:
# Handling দাঁড়ি problem 2.0
d = ' ।'
test_df['text'] = test_df["text"].str.replace(d, "।", regex=True)
test_dff['text'] = test_dff["text"].str.replace(d, "।", regex=True)

In [1101]:
# ডা
d = 'ডা\.'

test_dff['text'] = test_dff["text"].str.replace(d, "ডাক্তার", regex=True)

In [1102]:
# ড
d = 'ড\.'

test_dff['text'] = test_dff["text"].str.replace(d, "ডক্টর", regex=True)

In [1103]:
# স.
d = '\(স\.\)'
test_dff['text'] = test_dff["text"].str.replace(d, "সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম", regex=True)

In [1104]:
# সা.
d = '\(সা\.\)'
test_dff['text'] = test_dff["text"].str.replace(d, "সাল্লাল্লাহু আলাইহি ওয়া সাল্লাম", regex=True)

In [1105]:
# র.
d = '\(র\.\)'
test_dff['text'] = test_dff["text"].str.replace(d, "রাদিআল্লাহু আনহু", regex=True)

In [1106]:
# রা.
d = '\(রা\.\)'
test_dff['text'] = test_dff["text"].str.replace(d, "রাদিআল্লাহু আনহু", regex=True)

In [1107]:
# রহ.
d = '\(রহ\.\)'
test_dff['text'] = test_dff["text"].str.replace(d, "রহমাতুল্লাহি আলাইহি", regex=True)

In [1108]:
# অব.
d = '\(অব\.\)'

test_dff['text'] = test_dff["text"].str.replace(d, "অবসরপ্রাপ্ত", regex=True)

In [1109]:
# মো. 
d = 'মো\.'

test_dff['text'] = test_dff["text"].str.replace(d, "মোহাম্মদ", regex=True)

In [1110]:
# মু.
d = 'মু\.'

test_dff['text'] = test_dff["text"].str.replace(d, "মুহম্মদ", regex=True)

In [1111]:
# মু.
d = 'মুহ\.'

test_dff['text'] = test_dff["text"].str.replace(d, "মুহম্মদ", regex=True)

In [1112]:
# আ.
d = 'আ\.'

test_dff['text'] = test_dff["text"].str.replace(d, "আব্দুল", regex=True)

In [1113]:
# আওয়ামী লীগ
d = 'আ’'

test_dff['text'] = test_dff["text"].str.replace(d, "আওয়ামী ", regex=True)

In [1086]:
replace_numerals_with_words('০৭/২০০৭')

'NotValid'

In [1219]:
x = 15351
test_dff['text'][x]

'এ বিষয়ে শুক্রবার হাইকমিশনার মুহম্মদ শহীদুল ইসলাম যুগান্তরকে বলেন, কোনোভাবেই দালালদের হাইকমিশনে ঘেঁষতে দেয়া হচ্ছে না।'

In [1220]:
test_df['text'][x]

'এ বিষয়ে শুক্রবার হাইকমিশনার মুহ. শহীদুল ইসলাম যুগান্তরকে বলেন, কোনোভাবেই দালালদের হাইকমিশনে ঘেঁষতে দেয়া হচ্ছে না।'

In [1117]:
f = 'NotValid'
test_dff[lambda x: x["text"].str.contains(f)]

,row_id_column_name,text
1916,1916,দ্রবণটির মোলারিটি কত?. . .. . NotValid দশমিক তিন।
3618,3618,যা তার ক্যারিয়ার সেরা টাইমিংয়ের চেয়েও NotValid...
7212,7212,কাট করতে হয় কীভাবে?ক. যোগ খ. যোগ গ. NotValid য...
9339,9339,সময় : সকাল NotValid প্রয়োজন : দুই কপি পাসপোর্ট...
9620,9620,"সংশ্লিষ্ট আইনজীবীসহ একাধিক সূত্র জানায়, বিবিধ ..."
9698,9698,টুর্নামেন্টে বাংলাদেশের সাদ আলী সিদ্দিকী NotVa...
10370,10370,"পাকিস্তান প্রথম ইনিংস তিন/এক (সামি আসলাম তিন, ..."


In [2]:
# Sort by length
index = test_df["text"].str.len().sort_values(ascending=False).index
test_df = test_df.reindex(index)

In [480]:
# Remove English alphanumeric values
alpha_pat = "[a-zA-z0-9]"

test_df["text"] = test_df["text"].str.replace(alpha_pat, "", regex=True)

In [13]:
from transformers import pipeline
model_id = "iit-eight/checkpoint-36000/"
pipe = pipeline("text2text-generation", model=model_id, device=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
%%time
texts = test_df["text"].tolist()
ipas = pipe(texts, max_length=128, batch_size=16)
ipas = [ipa["generated_text"] for ipa in ipas]

CPU times: total: 50min 6s
Wall time: 51min 14s


In [6]:
ipas

['ecʰɐɽɐ cɔttogɾɐm-kɔksbɐɟɐɾ, dʱɐkɐ-cɔttogɾɐm, ɾɐŋguniʲɐ-cɔttogɾɐm, kʰɐgɽɐcʰoɽi-pʰotikcʰoɽi, bɐ̃ʃkʰɐli-ɐnoʷɐɾɐ, ɾɐmgɔɽ-hẽʲɐko-bɐɾo͡i̯ʲɐɾ hɐt o cɔttogɾɐmeɾ upo',
 'cɛmpiʲon hoʷɐ ud̪d̪oggulo hɔl- ʃebɐ pɔd̪d̪ʱot̪i ʃɔhoɟikɔɾon-espi͡e̯s, kɛtɐgoɾi-sɽo, poɾibeʃ od̪ʱid̪ɔpʰt̪ɔɾeɾ ɔnɔlɐ͡i̯n cʰɐɽpɔt̪ɾo ("kɛtɐgoɾi-sɽo, eboŋ kɾiʃɔkeɾ ɟɐn',
 'mɔnt̪ɾi bɔlen, bɔɾt̪omɐne beʃoɾkɐɾi poɾɟɐʲe cikit̪ʃɐʃebɐ kɐɾɟokɾom ‘d̪d̪o medikel pɾɐktiʃ ɛnd pɾɐ͡i̯bʱet klinik ɛnd lɛbɔɾetoɾiɟ,ɾeguleʃon, ɔɾdinnɐnʃo-d̪ɔɾeleʃɔn onuɟɐʲi poɾicɐlit̪o hoccʰe।',
 'ʃilpo mɔnt̪ɾonɐlɔʲ eboŋ i͡u̯ɾopiʲɐn i͡u̯niʲoneɾ ʃɔhɐʲt̪ɐʲ bɐŋlɐd̪eʃ gɐɾmentɔʃ eksesɾiɟ ɛnd pɛkeɟiŋ mɛnupʰɛkcɐɾɐɾs ɛnd ekspoɾtɐɾs ɛʃoʃiʲeʃon obiɟi͡e̯pi͡e̯me, e ʃeminɐɾ ɐ',
 'gʱuɾbot̪e bo͡i̯ʃɐkʰi bʱɾomon pɛkeɟe d̪ɐɾɟiliŋ-miɾikɐlimpɔŋ, kolkɐt̪ɐ-d̪illi-ʃimɔlɐ-mɐnɐli, kɐtʰmɐndu-nɐgoɾkot, bɐli, ind̪oneʃiʲɐ, bʱutɐn, mɐlod̪d̪ipʃɔho ɐɾɔ͡o̯ beʃ kɔʲekti st̪ʰɐne',
 'upoɟelɐɾ ɟoliʃɐ ʃɐmimiʲɐ d̪ɐkʰil mɐd̪ɾɐʃɐɾ ʃɔhokɐɾi ʃikkʰikɐ ("ʃɔmɐɟbiggɐn/ moʃɐ. bebi ɐkt̪ɐɾ, indikʃo nɔŋ-mɔbɔɾt̪o-bɔ

In [7]:
test_df

,row_id_column_name,text
13704,13704,"এছাড়া চট্টগ্রাম-কক্সবাজার, ঢাকা-চট্টগ্রাম, রাঙ..."
10739,10739,চ্যাম্পিয়ন হওয়া উদ্যোগগুলো হল- সেবা পদ্ধতি সহজ...
6971,6971,"মন্ত্রী বলেন, বর্তমানে বেসরকারি পর্যায়ে চিকিৎস..."
11358,11358,শিল্প মন্ত্রণালয় এবং ইউরোপিয়ান ইউনিয়নের সহায়তা...
6527,6527,ঘুরবতে বৈশাখী ভ্রমণ প্যাকেজে দার্জিলিং-মিরিককা...
...,...,...
11180,11180,জয় ৪ ম্যাচে।
7867,7867,আজ ৪ আষাঢ়।
11116,11116,৫৯ জন আহত।
13549,13549,- একা একা।


In [8]:
test_df["ipa"] = ipas
test_df = test_df.sort_index()
test_df.head()

,row_id_column_name,text,ipa
0,0,বিশেষ অতিথি এফএম ইকবাল বিন আনোয়ার (ডন) অ্যাডিশ...,"biʃeʃ ot̪it̪i epʰem ikbɐl bin ɐnoʷɐɾ ‘dɔn, ɛdi..."
1,1,এ নিয়ে বিবাদে ২০১৫ সালের ২ জুন রাত সাড়ে ১১টায় ...,e niʲe bibɐd̪e septembɔɾti sɐleɾ bɐ͡i̯ ɟun ɾɐt...
2,2,আজ থেকে ১৪ বছর আগে তিনি চলে গেছেন না ফেরার দেশে।,ɐɟ t̪ʰeke ʲcʰoɾ cʰɔboɾ ɐge t̪ini cɔle gecʰen n...
3,3,নিহত ব্যক্তি কুতপালং টালের ই-২ ব্লকের আবুল বাছ...,nihɔt̪o bekt̪i kut̪pɐloŋ tɐleɾ i-e͡i̯ blokeɾ ɐ...
4,4,সংক্ষিপ্ত স্কোরশ্রীলংকা প্রথম ইনিংস ৪৮২ (করুনা...,ʃɔŋkkʰipt̪o skoɾɔsɾilŋkɐ pɾot̪ʰom iniŋʃo cɐ͡i̯...


In [9]:
test_df = test_df.drop('text', axis = 1)

In [10]:
test_df.to_csv("submission_jup_eight_three.csv", index = None)